In [1]:
import ee
import geemap

ee.Initialize()

In [2]:
def get_s2_collection(aoi: ee.Geometry, start_date: str, end_date: str) -> ee.ImageCollection:
    """Get Sentinel-1 GRD data for a given area and time period.

    Args:
        aoi (ee.Geometry): Area of interest.
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.

    Returns:
        ee.ImageCollection: Sentinel-1 GRD data.
    """
    rsd = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20))
        
    )
    return rsd

In [3]:
def compute():
    for year in range(2017, 2022):
        aoi = ee.FeatureCollection("projects/yk-wetland-mapping/assets/yukon-bounds")
        start_mm_dd = f'{year}-06-20'
        end_mm_dd = f'{year}-09-22'
        dataset = get_s2_collection(aoi, start_mm_dd, end_mm_dd)
        yield dataset.median(), str(year)

s2_collections = compute()

In [8]:
Map = geemap.Map()
vis = {'bands': ['B4', 'B3', 'B2'],'min': 0, 'max': 3000}
dataset, year = next(s2_collections)
Map.addLayer(dataset, vis, f'S2 (B4, B3, B2): {year}')
Map.addLayerControl()

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…